# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.
    

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

In [1]:

import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [2]:
tudo = pd.ExcelFile('bobs.xlsx')                #Arquivo
dados = pd.read_excel(tudo, 'Treinamento')      #Planilha Treinamento
teste = pd.read_excel(tudo, 'Teste')            #Planilha Teste

<h3> Limpando as mensagens:</h3>

In [3]:
for i in range(len(dados)):
    caract=[',',';','.','!','?',"(",")", ':', '"', "'"]      #caracteres que se desejam retirar
    palavras = dados.Treinamento[i].split()                 # criando lista de listas de palavras em cada mensagem
    letras=[]                                              #lista de lista de letras das palvras
    final=[]                                                #lista de mensagens no final do processo
    for x in palavras:                                      #passando palavra por palavra
        letrasporpalavra=[]                                #lista que irá conter as letras das palavras no for
        for a in range(len(x)):                            #passando letra por letra
            letrasporpalavra.append(x[a])                   #adicionando cada letra na lista
        letras.append(letrasporpalavra)                 #adicionando lista de lista com as letras
    for z in letras:                                    #passando item/lista por lista
        for w in z:                                     #dentro da lista, passando letra por letra
            if w in caract:                            #se estiver entre os caracteres
                d=z.index(w)                           #pega o indice desse caracter
                z[d] = ' '                             #substitui ele na lista por um espaço
    for q in range(len(letras)):                      #q vai passar pelos índices da lista letras (palavra por palavra)
        junto = ''.join(letras[q])                    #junto vai tranformar a lista de letras em palavras           
        if len(junto)!=0: 
            if junto[0] !='@' and junto[0:5]!='https' and junto!= 'rt' and junto[0:3]!='kkk':  #retira alguns outros itens
                final.append(junto)                 #adiciona na lista final
    for a in range(len(final)-1):                    #passando pelo índice de cada palavra
        if final[a] =='de' and final[a+1] == 'bobs':  #juntando de bobs em uma palavra só
            final[a]='debobs'
            indicepexcluir = a+1
    try:                                            #se houver "de bobs", exclui o "bobs"
        del final[indicepexcluir]
    except:                                        #se não, continua
        continue
    dados.Treinamento[i] = ' '.join(final)         #junta as palavras das mensagens e substitui as mensagens de dados.Treinamento



In [4]:
dados_relevante = dados[dados.relevancia=='r'].Treinamento
dados_irrelevante = dados[dados.relevancia=='i'].Treinamento

<h3>Calculando porcentagens:</h3>
<ul><li>Porcentagem de mensagens relevantes ou irrelevantes</li></ul>

In [5]:
quantidademsgrelev=0               #cria variável que conterá o número de palavras consideradas relevantes
quantidademsgirrelev=0              #cria variável que conterá o número de palavras consideradas irrelevantes
for i in dados.relevancia:          #passa na coluna relevancia de mwnsagem por mensagem
    if i=='r':                      #se for classificada como relevante adiciona 1 na quantidademsgrelev
        quantidademsgrelev+=1
    if i=='i':                      ##se for classificada como irrelevante adiciona 1 na quantidademsgirrelev
        quantidademsgirrelev+=1
        
quantidademsgrelev=(quantidademsgrelev/len(dados.relevancia)) *100       #tranforma para porcentagem em relação ao total de msg
quantidademsgirrelev=(quantidademsgirrelev/len(dados.relevancia))*100     #tranforma para porcentagem em relação ao total de msg


<ul><li>Número de palavras relevantes ou irrelevantes</li></ul>

In [6]:
rel = []                           #cria lista que ira conter todas as palavras contidas nas mensagens relevantes
for msg in dados_relevante:       #passa mensagem por mensagem
    palavras = msg.split()         #separa as palvras
    for x in palavras:             #adiciona as palavras na lista rel
        rel.append(x)
relevante = pd.DataFrame({"Treinamento":rel}).Treinamento.value_counts()/len(rel)*100   


irrel = []                       #cria lista que ira conter todas as palavras contidas nas mensagens irrelevantes
for msg in dados_irrelevante:    #passa mensagem por mensagem
    palavras = msg.split()       #separa as palvras
    for x in palavras:           #adiciona as palavras na lista rel
        irrel.append(x)
irrelevante = pd.DataFrame({"Treinamento":irrel}).Treinamento.value_counts()/len(irrel)*100

<ul><li>Calculando número de palavras diferentes</li></ul>

In [7]:
palavrasdiferentes=[]                           #cria uma lista que irá conter as palavras diferentes
for palavrarelevante in rel:                    #passa pelas palavras presentes na lista rel(palavras relevantes)
    for palavrairrelevante in irrel:            #passa pelas palavras presentes na lista irrel(palavras irrelevantes)
        if palavrarelevante!= palavrairrelevante:       # se as palavras forem diferentes
            if palavrarelevante not in palavrasdiferentes:           # se a palavra não estiver em palavrasdiferentes, adiciona
                palavrasdiferentes.append(palavrarelevante)
            if palavrairrelevante not in palavrasdiferentes:
                palavrasdiferentes.append(palavrairrelevante)            

In [8]:
relevan={}                                            #cria dicionário que conterá a probabilidade de cada palavra dado relevante
palavrasrele=[]                                       #cria lista que irá conter as palavras
for palavrarel in rel:                                #passa palavra por palavra em rel(lista de palavras relevantes)
    if palavrarel not in palavrasrele:                #se a palvra ainda não estiver em palavrasrele
        porcentagem=(relevante[palavrarel]+1)/(len(rel)+len(palavrasdiferentes))  #calcula sua probabilidade
        relevan[palavrarel]=porcentagem*100                      #adiciona no dicionario da forma 'palavra':probabilidade
        palavrasrele.append(palavrarel)                   #adiciona a palavra em palavrasrele



irrelevan={}                                       #cria dicionário que conterá a probabilidade de cada palavra dado irrelevante
palavrasirrele=[]                                  #cria lista que irá conter as palavras
for palavrairrel in irrel:                         #passa palavra por palavra em irrel(lista de palavras irrelevantes)
    if palavrairrel not in palavrasirrele:         #se a palvra ainda não estiver em palavrasirrele
        porcentagem=(irrelevante[palavrairrel]+1)/(len(irrel)+len(palavrasdiferentes))    #calcula sua probabilidade
        irrelevan[palavrairrel]=porcentagem*100    #adiciona no dicionario da forma 'palavra':probabilidade
        palavrasirrele.append(palavrairrel)        #adiciona a palavra em palavrasrele


<h2>Planilha Teste</h2>

<h3> Limpando as mensagens:</h3>

In [15]:
for i in range(len(teste)):
    caract=[',',';','.','!','?',"(",")", ':', '"', "'"]      #caracteres que se desejam retirar
    palavras = teste.Teste[i].split()                 # criando lista de listas de palavras em cada mensagem
    letras=[]                                              #lista de lista de letras das palvras
    final=[]                                                #lista de mensagens no final do processo
    for x in palavras:                                      #passando palavra por palavra
        letrasporpalavra=[]                                #lista que irá conter as letras das palavras no for
        for a in range(len(x)):                            #passando letra por letra
            letrasporpalavra.append(x[a])                   #adicionando cada letra na lista
        letras.append(letrasporpalavra)                 #adicionando lista de lista com as letras
    for z in letras:                                    #passando item/lista por lista
        for w in z:                                     #dentro da lista, passando letra por letra
            if w in caract:                            #se estiver entre os caracteres
                d=z.index(w)                           #pega o indice desse caracter
                z[d] = ' '                             #substitui ele na lista por um espaço
    for q in range(len(letras)):                      #q vai passar pelos índices da lista letras (palavra por palavra)
        junto = ''.join(letras[q])                    #junto vai tranformar a lista de letras em palavras           
        if len(junto)!=0: 
            if junto[0] !='@' and junto[0:5]!='https' and junto!= 'rt' and junto[0:3]!='kkk':  #retira alguns outros itens
                final.append(junto)                 #adiciona na lista final
    for a in range(len(final)-1):                    #passando pelo índice de cada palavra
        if final[a] =='de' and final[a+1] == 'bobs':  #juntando de bobs em uma palavra só
            print(final[a],final[a+1])
            final[a]='debobs'
            indicepexcluir = a+1
    try:                                            #se houver "de bobs", exclui o "bobs"
        del final[indicepexcluir]
    except:                                        #se não, continua
        continue
    teste.Teste[i] = ' '.join(final)         #junta as palavras das mensagens e substitui as mensagens de teste.Teste

In [10]:
teste.loc[: , 'Diferença de probabilidades']='x'
teste.loc[: , 'Classificação Digital']='z'
for i in range(len(teste)):
    palavras = teste.Teste[i].split()
    probabilidaderelev =quantidademsgrelev
    probabilidadeirrelev=quantidademsgirrelev
    for q in palavras:
        if q in relevan:
            probabilidaderelev*=relevan[q]
        else:
            probabilidaderelev*=1/(len(rel)+len(palavrasdiferentes))
        if q in irrelevan:    
            probabilidadeirrelev*= irrelevan[q]
        else:
            probabilidadeirrelev*= 1/(len(irrel)+len(palavrasdiferentes))
    if probabilidaderelev> probabilidadeirrelev:
        teste['Classificação Digital'][i]='r'
    else:
        teste['Classificação Digital'][i]='i'       
    probdifR=probabilidaderelev-probabilidadeirrelev
    
    teste['Diferença de probabilidades'][i]=probdifR
teste.loc[teste['Diferença de probabilidades'] < - (10**(-10)), 'Especifico']='mi'
teste.loc[teste['Diferença de probabilidades'] > - (10**(-10)), 'Especifico']='i'
teste.loc[teste['Diferença de probabilidades'] > - (10**(-20)), 'Especifico']='n'
teste.loc[teste['Diferença de probabilidades'] >  (10**(-20)), 'Especifico']='r'
teste.loc[teste['Diferença de probabilidades'] >  (10**(-10)), 'Especifico']='mr'
teste.head(36)        

,Teste,Manual,EspManual,Diferença de probabilidades,Classificação Digital,Especifico
0,só um lanche do ric ou coelho na minha vida,i,n,1.46438e-16,r,r
1,nos foi no bobs ainda comemos uma pizza 🍕👫,i,i,-1.64031e-14,i,i
2,minha mãe foi pro e trouxe um lanche do bobs p...,r,r,-4.49035e-18,i,i
3,eu e neandro deixando o bglh de julia no chão ...,i,mi,-1.76991e-23,i,n
4,que victor que vc no bobs — santos ❤ amigo,i,mi,1.13158e-18,r,r
5,criando forças p levantar ir no bobs comer alg...,i,i,-5.26104e-16,i,i
6,boa tarde ☉,i,mi,3.56386e-05,r,mr
7,minha mae ficou com de mim e mandou eu cmp lan...,i,n,-5.97873e-19,i,i
8,acabei de ver q big cascão tá 2 50 pelo bobs f...,i,n,-1.63089e-24,i,n
9,cheguei no topo do tava tudo sujo de lanche do...,i,i,1.00485e-34,r,n


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [11]:
pf=0
pv=0
nv=0
nf=0
for i in range(len(teste['Classificação Digital'])):
    if teste['Classificação Digital'][i] == 'i':
        if teste.Manual[i] == 'i':
            nv+=1
            
        else:
            nf+=1
    if teste['Classificação Digital'][i] == 'r':
        if teste.Manual[i] == 'r':
            pv+=1
        else:
            pf+=1
pf=(pf/len(teste['Classificação Digital']))*100
pv=(pv/len(teste['Classificação Digital']))*100
nv=(nv/len(teste['Classificação Digital']))*100
nf=(nf/len(teste['Classificação Digital']))*100



print(pf,pv,nv,nf)


33.0 21.0 37.5 8.5


In [12]:

lista=['mr','mi','n','r','i']
dic={}

for o in lista:
    dic[o]=0
for o in lista:
    for j in range(len(lista)):
        dic[o+lista[j]]=0
        
for i in range(len(teste.EspManual)):
        if teste.Especifico[i] == teste.EspManual[i]:
            dic[teste.Especifico[i]]=dic[teste.Especifico[i]]+1
        else:
            dic[teste.EspManual[i]+teste.Especifico[i]]=dic[teste.Especifico[i]+teste.EspManual[i]]+1
deletar=[]
for i in dic:
    if dic[i]==0:
        deletar.append(i)
for i in deletar:
    del(dic[i])
print(dic)
for i in dic:
    dic[i]=(dic[i]/len(teste.Especifico))*100
print()
print(dic)



{'mr': 8, 'mi': 6, 'n': 16, 'r': 6, 'i': 13, 'mrmi': 2, 'mrn': 7, 'mrr': 7, 'mimr': 3, 'min': 13, 'mir': 7, 'mii': 9, 'nmr': 6, 'nmi': 12, 'nr': 11, 'ni': 5, 'rmr': 8, 'rmi': 6, 'rn': 10, 'ri': 1, 'imr': 1, 'imi': 8, 'in': 6, 'ir': 2}

{'mr': 4.0, 'mi': 3.0, 'n': 8.0, 'r': 3.0, 'i': 6.5, 'mrmi': 1.0, 'mrn': 3.5000000000000004, 'mrr': 3.5000000000000004, 'mimr': 1.5, 'min': 6.5, 'mir': 3.5000000000000004, 'mii': 4.5, 'nmr': 3.0, 'nmi': 6.0, 'nr': 5.5, 'ni': 2.5, 'rmr': 4.0, 'rmi': 3.0, 'rn': 5.0, 'ri': 0.5, 'imr': 0.5, 'imi': 4.0, 'in': 3.0, 'ir': 1.0}


In [13]:
p = teste.Especifico.value_counts(True)*100
p

n     36.0
r     20.5
mr    19.5
i     13.0
mi    11.0
Name: Especifico, dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
